In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
from tqdm import tqdm_notebook as tqdm
import shutil
import numpy as np
import pandas_profiling
import datetime


pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_info_rows", 200)

In [2]:
import sephora_scrape as scrape
%reload_ext autoreload
%autoreload 2

In [3]:
df_sephora= pd.read_csv("df_sephora.csv")
df_sephora

,id,name,brand_id,brand_name,short_desc,long_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,is_natural_organic,rating,review_count,user_favorites,categories,url,image_url,ingredients,suggested_usage,brand_long,has_variants,has_categories,has_ingredients,has_suggested_usage,is_rated,cologne,men,fragrance,perfume,women,lotions_and_oils,rollerballs_and_travel_size,perfume_gift_sets,value_and_gift_sets,lipstick,lip,makeup,lip_balms_and_treatments,lip_treatments,mascara,eye,lip_gloss,eyebrow,body_sprays_and_deodorant,bath_and_shower,cologne_gift_sets,body_mist_and_hair_mist,concealer,face,lip_stain,deodorant_and_antiperspirant,body_care,bath_and_body,face_primer,foundation,tinted_moisturizer,face_serums,treatments,skincare,mists_and_essences,moisturizers,highlighter,liquid_lipstick,lip_sets,face_sets,setting_spray_and_powder,aftershave,shaving,eye_creams_and_treatments,eye_care,face_masks,masks,face_wash_and_cleansers,cleansers,exfoliators,makeup_removers,eyeliner,face_brushes,makeup_brushes_and_applicators,makeup_palettes,eye_palettes,bb_and_cc_creams,blush,cheek,facial_peels,eyelash_curlers,accessories,eye_sets,mini_size,bronzer,face_oils,contour,lip_liner,eyeshadow,face_wash,moisturizer_and_treatments,toners,body_lotions_and_body_oils,body_moisturizers,scrub_and_exfoliants,self_tanners,face_sunscreen,sun_care,blemish_and_acne_treatments,facial_cleansing_brushes,high_tech_tools,for_face,color_correct,skincare_sets,decollete_and_neck_creams,hand_cream_and_foot_cream,candles_and_home_scents,night_creams,sheet_masks,eye_brushes,beauty_supplements,wellness,facial_rollers,body_sunscreen,cellulite_and_stretch_marks,for_body,body_wash_and_shower_gel,nail,other_needs,shampoo_and_conditioner,hair,hair_styling_and_treatments,conditioner,shampoo,eye_masks,eye_primer,mirrors_and_sharpeners,tools_and_brushes,cheek_palettes,face_wipes,tweezers_and_eyebrow_tools,hair_accessories,hair_tools,gift_cards,false_eyelashes,brush_sets,brush_cleaners,makeup_and_travel_cases,hair_removal,lip_brushes,sponges_and_applicators,makeup_bags_and_travel_cases,hair_masks,curling_irons,hair_straighteners_and_flat_irons,hair_dryers,dry_shampoo,hair_brushes_and_combs,scalp_and_hair_treatments,bath_soaks_and_bubble_bath,hair_styling_products,blotting_papers,hair_primers,hair_oil,teeth_whitening,anti_aging,travel_size,facial_men,candles,diffusers,holistic_wellness,leave_in_conditioner,hair_spray,color_care,best_for,just_arrived,gifts_her,gifts_men,gifts_them,gifts_teens,gift_sets_men,gift_sets,editors_picks,luxury_gifts,sephora_exclusives,perfume_men,grooming_men,hair_men,personal_care_men,just_arrived_men,best_seller_men
0,P180726,Pour Homme II,1063,Gucci,"An alluring fragrance heightened by a sensual, spicy soul, Gucci Pour Homme II is the essence of young, masculine sensuality.","An alluring fragrance heightened by a sensual, spicy soul, Gucci Pour Homme II is the essence of young, masculine sensuality. The embodiment of relaxed refinement for the modern-minded man whose style is distinctive and effortless, this fresh, spicy, woody, scent is filled with playful attitude and charisma. Notes: Bergamot, Violet Leaves, Cinnamon, Pimento, Black Tea, Myrrh, Tobacco Leaves, White Musk, Olive Wood. Style: Seductive. Masculine. Free-spirited.",997338,3.3 oz/ 98 mL Eau de Toilette Spray,92.00,Size + Concentration + Formulation,NaN,92.00,92.00,0,0,4.81,114,2030,"{'cat60150': 'Cologne', 'cat1230040': 'Men', 'cat160006': 'Fragrance'}",https://www.sephora.com/product/pour-homme-ii-P180726?skuId=997338,https://www.sephora.com/productimages/sku/s997338-main-zoom.jpg,"Alcohol Denat., Fragrance, Water, Ethylhexyl Methoxycinnamate, Diethylamino Hydroxybenzoyl Hexyl Benzoate, BHT, Benzyl Salicylate, Linalool, Butylphenyl Methylpropional, Hydroxycitronellal, Limonene, Hexyl Cinnamal, Citronellol, Alpha-Isomethyl Ionone, Geraniol, Hydroxyisohexyl 3-Cyclohexene Carboxaldehyde, Citral, Farnesol, Isoeugenol, Anise Alcohol, Benzyl Benzoate, CI 1720

In [4]:
reviews_folder = "/Users/valmadrid/DataScienceBootcamp/Projects/Gift Recommendation/Gift-Recommendation/reviews/"
if not os.path.exists(reviews_folder):
    print("check file path")

In [5]:
# os.mkdir(reviews_folder)

In [11]:
def get_review_json(filename, folder_path):
    
    start_date = int(datetime.datetime(2015, 6, 25, 23, 59, 59).timestamp())
    end_date = int(datetime.datetime(2020, 6, 25, 23, 59, 59).timestamp())
    
    for offset in range(-1, 100):
        
        prefix = "https://api.bazaarvoice.com/data/reviews.json?apiversion=5.4&passkey=rwbw526r2e7spptqd2qzbkp7"
        filter_id = "&Filter=ProductId%3A{}".format(filename)
        filter_time = "&filter=SubmissionTime:gt:{}&filter=SubmissionTime:lt:{}".format(start_date, end_date)
        includes = "&Include=Products%2CComments&Stats=Reviews"
        page = "&Sort=SubmissionTime:desc&Limit=100&Offset={}".format(offset*100)
        
        if offset == -1:
            full_url = prefix + filter_id + includes + filter_time
            
            try:
                r = requests.get(full_url)
                reviews_data = json.loads(r.content)
                with open(folder_path + filename + "_stats"+ ".json",
                          "w") as outfile:
                    json.dump(reviews_data, outfile, indent=2)
            except:
                print("Error downloading ", filename)
                break
            
        else:
            
            full_url = prefix+filter_id+filter_time+page
            
            try:
                r = requests.get(full_url)
                reviews_data = json.loads(r.content)
                with open(folder_path + filename + "_" + str(offset) + ".json",
                          "w") as outfile:
                    json.dump(reviews_data, outfile, indent=2)

                if reviews_data["TotalResults"] - 100 * (offset + 1) > 0:
                    continue
                else:
                    break
            except:
                print("Error downloading ", filename)
                break
    r.close()

In [14]:
downloaded_json_reviews = list(set([file.replace("_0.json", "") for file in os.listdir(reviews_folder) if file.endswith("_0.json")]))
downloaded_json_reviews[:10]

['P419634',
 'P387228',
 'P440929',
 'P403328',
 'P393356',
 'P456412',
 'P420168',
 'P452412',
 'P457655',
 'P429661']

In [15]:
for product_ref in tqdm(list(df_sephora.id)):
    if product_ref in downloaded_json_reviews:
        continue
    else:
        get_review_json(product_ref, reviews_folder)